In [2]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.9 GB  | Proc size: 143.0 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [0]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import nltk

In [4]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

    100% |████████████████████████████████| 993kB 18.6MB/s 


In [0]:
#train.csv

# Download a file based on its file ID.
# https://drive.google.com/open?id=1A3BLkSn7GOwR7J2ccoIoDoQVOXLeytyR

file_id = '1A3BLkSn7GOwR7J2ccoIoDoQVOXLeytyR' #https://drive.google.com/open?id=1R72cIA1deLDdg0aULjcl5OR6tsv-DRaq
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('sample_submission.csv')

In [6]:
submission = pd.read_csv('sample_submission.csv', encoding = "ISO-8859-1")
submission.head()

,id,Sent_ID,tag
0,4543834,191283,O
1,4543835,191283,O
2,4543836,191283,O
3,4543837,191283,O
4,4543838,191283,O


In [7]:
submission.shape

(2994463, 3)

In [0]:
#test.csv

file_id = '1A74kdE3pv_WvHJwuEbeF2TlYk4irXlis' #https://drive.google.com/open?id=1R72cIA1deLDdg0aULjcl5OR6tsv-DRaq
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('test.csv')

In [9]:
test_df = pd.read_csv('test.csv', encoding = "ISO-8859-1")
test_df.head(5)

,id,Doc_ID,Sent_ID,Word
0,4543834,30001,191283,CCCVA
1,4543835,30001,191283,","
2,4543836,30001,191283,MANOVA
3,4543837,30001,191283,","
4,4543838,30001,191283,my


In [10]:
test_df.shape

(2994463, 4)

In [0]:
#train.csv

# Download a file based on its file ID.

file_id = '1R72cIA1deLDdg0aULjcl5OR6tsv-DRaq' #https://drive.google.com/open?id=1R72cIA1deLDdg0aULjcl5OR6tsv-DRaq
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('train.csv')

In [12]:
train_df = pd.read_csv('train.csv', encoding = "ISO-8859-1")
train_df.head()

,id,Doc_ID,Sent_ID,Word,tag
0,1,1,1,Obesity,O
1,2,1,1,in,O
2,3,1,1,Low-,O
3,4,1,1,and,O
4,5,1,1,Middle-Income,O


In [13]:
train_df.isnull().sum()

id           0
Doc_ID       0
Sent_ID      0
Word       130
tag          0
dtype: int64

In [14]:
train_df = train_df.fillna(method='ffill')
print(train_df.Sent_ID.nunique(), train_df.Word.nunique(), train_df.tag.nunique())

test_df = test_df.fillna(method='ffill')
print(test_df.Sent_ID.nunique(), test_df.Word.nunique())

train_df['id_Sent_ID'] = train_df['id'] + train_df['Sent_ID']
test_df['id_Sent_ID'] = test_df['id'] + test_df['Sent_ID']

191282 184505 3
125840 139891


In [15]:
train_df.head(2)

,id,Doc_ID,Sent_ID,Word,tag,id_Sent_ID
0,1,1,1,Obesity,O,2
1,2,1,1,in,O,3


## Data Preprocessing

In [16]:
train_df.isnull().sum()

id            0
Doc_ID        0
Sent_ID       0
Word          0
tag           0
id_Sent_ID    0
dtype: int64

We have 191,282 sentences that contain 184,505 unique words and tagged by 3 tags.

The tags are not evenly distributed.

In [17]:
train_df.groupby('tag').size().reset_index(name='counts')

,tag,counts
0,B-indications,53003
1,I-indications,44624
2,O,4446206


In [18]:
test_df.isnull().sum()

id            0
Doc_ID        0
Sent_ID       0
Word          0
id_Sent_ID    0
dtype: int64

In [0]:
# import nltk
# nltk.download('all')

In [20]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [0]:
train_df['POS_full'] = nltk.pos_tag(train_df['Word']) 
test_df['POS_full'] = nltk.pos_tag(test_df['Word']) 

In [26]:
train_df.head(5)

,id,Doc_ID,Sent_ID,Word,tag,id_Sent_ID,POS_full,POS
0,1,1,1,Obesity,O,2,"(Obesity, NN)",NN
1,2,1,1,in,O,3,"(in, IN)",IN
2,3,1,1,Low-,O,4,"(Low-, NNP)",NNP
3,4,1,1,and,O,5,"(and, CC)",CC
4,5,1,1,Middle-Income,O,6,"(Middle-Income, JJ)",JJ


In [27]:
test_df.head(5)

,id,Doc_ID,Sent_ID,Word,id_Sent_ID,POS_full
0,4543834,30001,191283,CCCVA,4735117,"(CCCVA, NNP)"
1,4543835,30001,191283,",",4735118,"(,, ,)"
2,4543836,30001,191283,MANOVA,4735119,"(MANOVA, NNP)"
3,4543837,30001,191283,",",4735120,"(,, ,)"
4,4543838,30001,191283,my,4735121,"(my, PRP$)"


In [0]:
train_df['POS'] = train_df['POS_full'].apply(lambda x: x[1] )
test_df['POS'] = test_df['POS_full'].apply(lambda x: x[1] )

In [29]:
train_df.head(5)

,id,Doc_ID,Sent_ID,Word,tag,id_Sent_ID,POS_full,POS
0,1,1,1,Obesity,O,2,"(Obesity, NN)",NN
1,2,1,1,in,O,3,"(in, IN)",IN
2,3,1,1,Low-,O,4,"(Low-, NNP)",NNP
3,4,1,1,and,O,5,"(and, CC)",CC
4,5,1,1,Middle-Income,O,6,"(Middle-Income, JJ)",JJ


In [30]:
test_df.head(5)

,id,Doc_ID,Sent_ID,Word,id_Sent_ID,POS_full,POS
0,4543834,30001,191283,CCCVA,4735117,"(CCCVA, NNP)",NNP
1,4543835,30001,191283,",",4735118,"(,, ,)",","
2,4543836,30001,191283,MANOVA,4735119,"(MANOVA, NNP)",NNP
3,4543837,30001,191283,",",4735120,"(,, ,)",","
4,4543838,30001,191283,my,4735121,"(my, PRP$)",PRP$


In [0]:
# train_df = train_df.drop(['Doc_ID', 'POS_full','id'], axis= 1)
# test_df = test_df.drop(['Doc_ID', 'POS_full','id'], axis= 1)

The following code transform the text date to vector using DictVectorizer and then split to train and test sets.

In [32]:
train_df.id_Sent_ID.nunique(), train_df.Word.nunique(), train_df.POS.nunique()

(4543833, 184505, 44)

In [33]:
test_df.id_Sent_ID.nunique(), test_df.Word.nunique(), test_df.POS.nunique()

(2994463, 139891, 44)

In [34]:
test_df.isnull().sum()

id            0
Doc_ID        0
Sent_ID       0
Word          0
id_Sent_ID    0
POS_full      0
POS           0
dtype: int64

In [35]:
test_df.columns

Index(['id', 'Doc_ID', 'Sent_ID', 'Word', 'id_Sent_ID', 'POS_full', 'POS'], dtype='object')

In [0]:
test_df.to_csv('test_df_mod.csv')

In [37]:
train_df.head(3)

,id,Doc_ID,Sent_ID,Word,tag,id_Sent_ID,POS_full,POS
0,1,1,1,Obesity,O,2,"(Obesity, NN)",NN
1,2,1,1,in,O,3,"(in, IN)",IN
2,3,1,1,Low-,O,4,"(Low-, NNP)",NNP


In [0]:
train_df.to_csv('train_df_mod.csv')

### Time Consuming - Hence you can start here!!

In [0]:

file_id = '1FM9w8uPSpxult-rIl6HqqASqJroAINs1'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('train_df_mod.csv')


file_id = '1b75Rzb_XX32mzfoFFKkCFUZBGW419U30'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('test_df_mod.csv')


train_df = pd.read_csv('train_df_mod.csv')
test_df = pd.read_csv('test_df_mod.csv')

In [40]:
train_df.head(3)

,Unnamed: 0,id,Doc_ID,Sent_ID,Word,tag,id_Sent_ID,POS_full,POS
0,0,1,1,1,Obesity,O,2,"('Obesity', 'NN')",NN
1,1,2,1,1,in,O,3,"('in', 'IN')",IN
2,2,3,1,1,Low-,O,4,"('Low-', 'NNP')",NNP


In [41]:
train_df.tail(5)

,Unnamed: 0,id,Doc_ID,Sent_ID,Word,tag,id_Sent_ID,POS_full,POS
4543828,4543828,4543829,30000,191282,translational,O,4735111,"('translational', 'JJ')",JJ
4543829,4543829,4543830,30000,191282,or,O,4735112,"('or', 'CC')",CC
4543830,4543830,4543831,30000,191282,post-translational,O,4735113,"('post-translational', 'JJ')",JJ
4543831,4543831,4543832,30000,191282,level,O,4735114,"('level', 'NN')",NN
4543832,4543832,4543833,30000,191282,.,O,4735115,"('.', '.')",.


In [42]:
test_df.head(3)

,Unnamed: 0,Sent_ID,Word,id_Sent_ID,POS
0,0,191283,CCCVA,4735117,NNP
1,1,191283,",",4735118,","
2,2,191283,MANOVA,4735119,NNP


In [43]:
train_df = train_df[['Sent_ID', 'Word', 'tag', 'POS']]
print(train_df.shape)


test_df = test_df[['Sent_ID', 'Word',  'POS']]
print(test_df.shape)

(4543833, 4)
(2994463, 3)


In [44]:
words = list(set(train_df["Word"].values))
n_words = len(words); n_words

184505

### We start by writing a small class to retrieve a sentence from the dataset.

In [0]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
    
    def get_next(self):
#         try:
#             s = self.data[self.data["Sent_ID"] == "Sentence: {}".format(self.n_sent)]
#             self.n_sent += 1
            return self.data["Word"].values.tolist(), self.data["POS"].values.tolist(), self.data["tag"].values.tolist()    
#         except:
#             self.empty = True
#             return None, None, None

In [0]:
getter = SentenceGetter(train_df)

In [0]:
sent, pos, tag = getter.get_next()

In [48]:
### This is how a sentence looks.
print(sent); print(pos); print(tag)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
from sklearn.base import BaseEstimator, TransformerMixin


class MemoryTagger(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y):
        '''
        Expects a list of words as X and a list of tags as y.
        '''
        voc = {}
        self.tags = []
        for x, t in zip(X, y):
            if t not in self.tags:
                self.tags.append(t)
            if x in voc:
                if t in voc[x]:
                    voc[x][t] += 1
                else:
                    voc[x][t] = 1
            else:
                voc[x] = {t: 1}
        self.memory = {}
        for k, d in voc.items():
            self.memory[k] = max(d, key=d.get)
    
    def predict(self, X, y=None):
        '''
        Predict the the tag from memory. If word is unknown, predict 'O'.
        '''
        return [self.memory.get(x, 'O') for x in X]

In [50]:
tagger = MemoryTagger()
tagger.fit(sent, tag)
print(tagger.predict(sent))
tagger.tags

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



['O', 'B-indications', 'I-indications']

In [51]:
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.metrics import classification_report


words = train_df["Word"].values.tolist()
tags = train_df["tag"].values.tolist()


pred = cross_val_predict(estimator=MemoryTagger(), X=words, y=tags, cv=5)

report = classification_report(y_pred=pred, y_true=tags)
print(report)

               precision    recall  f1-score   support

B-indications       0.72      0.40      0.52     53003
I-indications       0.59      0.37      0.46     44624
            O       0.99      1.00      0.99   4446206

    micro avg       0.98      0.98      0.98   4543833
    macro avg       0.77      0.59      0.66   4543833
 weighted avg       0.98      0.98      0.98   4543833



In [0]:
class SentenceGetter_test_df(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
    
    def get_next(self):
#         try:
#             s = self.data[self.data["Sent_ID"] == "Sentence: {}".format(self.n_sent)]
#             self.n_sent += 1
            return self.data["Word"].values.tolist(), self.data["POS"].values.tolist()    
#         except:
#             self.empty = True
#             return None, None, None

In [0]:
getter_test = SentenceGetter_test_df(test_df)

In [0]:
sent, pos = getter_test.get_next()

In [55]:
print(sent); print(pos);

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
y_pred = tagger.predict(sent)

In [57]:
np.unique(y_pred, return_counts = True)

(array(['B-indications', 'I-indications', 'O'], dtype='<U13'),
 array([  19726,   18011, 2956726]))

In [58]:
submission.head()

,id,Sent_ID,tag
0,4543834,191283,O
1,4543835,191283,O
2,4543836,191283,O
3,4543837,191283,O
4,4543838,191283,O


In [0]:
submission['tag'] = pd.DataFrame(y_pred)

In [60]:
submission['tag'].value_counts()

O                2956726
B-indications      19726
I-indications      18011
Name: tag, dtype: int64

In [0]:
submission.to_csv('submission_baseline_model.csv', index= False)

In [0]:
from google.colab import files
files.download("/content/submission_baseline_model.csv")

In [0]:
#Source of the code

# https://www.depends-on-the-definition.com/introduction-named-entity-recognition-python/

## A simple machine learning approach

In [64]:
from sklearn.ensemble import RandomForestClassifier

def feature_map(word):
    '''Simple feature map.'''
    return np.array([word.istitle(), word.islower(), word.isupper(), len(word),
                     word.isdigit(),  word.isalpha()])
  
words = [feature_map(w) for w in train_df["Word"].values.tolist()]

pred = cross_val_predict(RandomForestClassifier(n_estimators=20),
                         X=words, y=tags, cv=5)

report = classification_report(y_pred=pred, y_true=tags)
print(report)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


               precision    recall  f1-score   support

B-indications       0.44      0.00      0.00     53003
I-indications       0.00      0.00      0.00     44624
            O       0.98      1.00      0.99   4446206

    micro avg       0.98      0.98      0.98   4543833
    macro avg       0.47      0.33      0.33   4543833
 weighted avg       0.96      0.98      0.97   4543833



In [0]:
words_test_rf = [feature_map(w) for w in test_df["Word"].values.tolist()]
model_rf = RandomForestClassifier(n_estimators=20)
model_rf.fit(words, tags)
pred = model_rf.predict(words_test_rf)


In [66]:
submission['tag'] = pd.DataFrame(pred)
submission['tag'].value_counts()

O                2994458
B-indications          5
Name: tag, dtype: int64

## Randomforest is not good

In [0]:
from sklearn.preprocessing import LabelEncoder

class FeatureTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        self.memory_tagger = MemoryTagger()
        self.tag_encoder = LabelEncoder()
        self.pos_encoder = LabelEncoder()
        
    def fit(self, X, y):
        words = X["Word"].values.tolist()
        self.pos = X["POS"].values.tolist()
        tags = X["tag"].values.tolist()
        self.memory_tagger.fit(words, tags)
        self.tag_encoder.fit(tags)
        self.pos_encoder.fit(self.pos)
        return self
    
    def transform(self, X, y=None):
        def pos_default(p):
            if p in self.pos:
                return self.pos_encoder.transform([p])[0]
            else:
                return -1
        
        pos = X["POS"].values.tolist()
        words = X["Word"].values.tolist()
        out = []
        for i in range(len(words)):
            w = words[i]
            p = pos[i]
            if i < len(words) - 1:
                wp = self.tag_encoder.transform(self.memory_tagger.predict([words[i+1]]))[0]
                posp = pos_default(pos[i+1])
            else:
                wp = self.tag_encoder.transform(['O'])[0]
                posp = pos_default(".")
            if i > 0:
                if words[i-1] != ".":
                    wm = self.tag_encoder.transform(self.memory_tagger.predict([words[i-1]]))[0]
                    posm = pos_default(pos[i-1])
                else:
                    wm = self.tag_encoder.transform(['O'])[0]
                    posm = pos_default(".")
            else:
                posm = pos_default(".")
                wm = self.tag_encoder.transform(['O'])[0]
            out.append(np.array([w.istitle(), w.islower(), w.isupper(), len(w), w.isdigit(), w.isalpha(),
                                 self.tag_encoder.transform(self.memory_tagger.predict([w]))[0],
                                 pos_default(p), wp, wm, posp, posm]))
        return out

In [68]:
train_df.head(3)

,Sent_ID,Word,tag,POS
0,1,Obesity,O,NN
1,1,in,O,IN
2,1,Low-,O,NNP


In [0]:
from sklearn.pipeline import Pipeline

pred = cross_val_predict(Pipeline([("feature_map", FeatureTransformer()), 
                                   ("clf", RandomForestClassifier(n_estimators=20, n_jobs=3))]),
                         X=train_df, y=tags, cv=5)

report = classification_report(y_pred=pred, y_true=tags)
print(report)

In [0]:
# In Pipeline we will modify the text and train RandomForestClassifier 
classifier = Pipeline([("feature_map", FeatureTransformer()), 
                                   ("clf", RandomForestClassifier(n_estimators=20, n_jobs=3))])
classifier.fit(train_df, tags)
y_prd=classifier.predict(test_df)

In [0]:
submission['tag'] = pd.DataFrame(y_prd)

In [0]:
from google.colab import files
files.download("/content/submission_baseline_rf_model.csv")